In [1]:
import os
import cv2
import shutil
import numpy as np
from PIL import Image
import shutil
from scipy.spatial.distance import cdist
import time
from imutils import paths
from matplotlib import pyplot as plt
import random
import math
from tqdm import tqdm


import zipfile
import os 
import cv2



import warnings
warnings.filterwarnings('ignore')
import shutil
import os
import pickle
import pandas as pd
import numpy as np
from numpy.matlib import repmat
import zipfile
from PIL import Image
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image

from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from imutils import paths



In [2]:

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# utils path
classifier_model_path = 'wheat_smc_t1_wo_decoder.h5'

smc_classes = pickle.load(open('wheat_smc_t1_class_names.json', 'rb'))['class_names']
print('for classification > {}, classes {}'.format(classifier_model_path, smc_classes))
smc_classes = ['Bad', 'Good']

for classification > wheat_smc_t1_wo_decoder.h5, classes ['ankur_wheat_bad', 'ankur_wheat_good']


In [10]:
#@title smc class

import tensorflow as tf
def my_init(shape, dtype=None):
    initializer = tf.keras.initializers.he_uniform(seed = 1)
    return initializer(shape, dtype=dtype)

def _no_loss(y_true, y_pred):
    '''
    loss function is not required for inference
    '''
    return np.float32(0)

classifier = load_model(classifier_model_path,  compile=False)

# classifier.compile(
#     optimizer=SGD(),
#     loss={
#         'decoder_o/p':_no_loss, # (uncomment only if AEC model)
#         'classifier_o/p': _no_loss
#     }
# )

def show(objs, titles, structure='LR', figsize=(10, 5), clims=False, axes=False, xlabels=False):
    '''
    utility function to show plots (subplots)
    '''
    plt.style.use('default')
    plt.figure(figsize=figsize)
    for i, (obj, name) in enumerate(zip(objs, titles)):
        if structure == 'LR': plt.subplot(1, len(titles), i+1)
        elif structure == 'TB': plt.subplot(len(titles), 1, i+1)
        plt.imshow(obj)
        if axes: plt.axis(axes[i])
        else: plt.axis('off')
        if xlabels: plt.xlabel(xlabels[i])
        if clims: plt.clim(clims[i])
        plt.title(name)
    plt.show()

def predict_seed_types(im_path, dsize=(448, 224)):#(448,224)for paddy
    '''
    predict from path.. (later)
    '''
    img = image.load_img(im_path, color_mode='rgb', target_size=dsize)
    img = image.img_to_array(img).astype('float32') / 255.0

    y = classifier.predict(np.expand_dims(img, axis=0))
    y = smc_classes[np.argmax(y)]
    return y
    

In [4]:
s = [ f.name for f in os.scandir('./e') if f.is_dir() ]

In [5]:
s

['wheat_kedar_lot-1']

In [8]:
cd ..

/home/anand/Documents/b


In [7]:
for x in s:
    
    os.mkdir(x)

In [ ]:
for x in s:
    inp_smc = 'e/'+x
    out_smc='s/'+x
    path = inp_smc
    new_path = out_smc
    
    img_paths = []
    main_types = []
    class_labels = []

    IGNORE = []

    # for class_ in sorted(os.listdir(path)):
    #     imgDir = '{}{}'.format(path, class_)
    #     print('* directory: {}'.format(imgDir))

    for imgFname in sorted(os.listdir(path)):
        if imgFname.endswith('.png'):
            #img_paths.append('{}/{}'.format(imgDir, imgFname))
            img_paths.append(os.path.join(path,imgFname))
            #class_labels.append(class_)

    print('Total samples: {}'.format(len(img_paths)))

    data = pd.DataFrame()
    data['img_path'] = img_paths
    # data['class_label'] = class_labels
    data = data.sample(frac=1.0, random_state=0)
    #data.tail()
    
    
    from shutil import copyfile, rmtree


    dir_ = os.path.join(new_path,"good") #path.replace('extracted', 'extracted_good_seeds')
    for clas in smc_classes:
      # other_dir = path.replace('extracted', 'extracted_other_seeds')
      #os.mkdir(clas)
      other_dir = os.path.join(new_path,"bad")



    try:
      rmtree(dir_)
      rmtree(other_dir)
    except: pass
    os.mkdir(dir_)

    os.mkdir(other_dir)
    [os.mkdir('{}/{}'.format(other_dir, class_)) for class_ in smc_classes]
    
    #%%time

    num_good_seeds = 0

    for i, im_path in enumerate(data['img_path']):
      smc_label = predict_seed_types(im_path)
      if smc_label == 'Good':
          num_good_seeds += 1
          print('{}/{} (Total good seeds: {}) Processing {}---------|'
                .format(i+1, len(data), num_good_seeds,im_path), end='\r')

          # save a copy to new dir_
          copyfile(im_path, im_path.replace(path, dir_))
      else:
          copyfile(im_path, '{}/{}/{}'.format(other_dir, smc_label, im_path.split('/')[-1]))



Total samples: 23869
